In [1]:
!pip install langchain-community langchain_anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.3/286.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.60
    Uninstalling langchain-core-0.3.60:
      Successfully uninstalled langchain-core-0.3.60


In [44]:
# ZERO-SHOT LEARNING

import openai
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": """
              Classifica il seguente feedback di un cliente in positivo,
              neutro o negativo.

              Esempio:
                - Feedback: "Il vostro ERP fornisce una reportistica troppo lenta"
                  Classificazione: Negativa

                - Feedback: "Il vostro ERP ha migliorato la nostra gestione dei lead, ma ha dei bug"
                  Classificazione: Positiva

                - Feedback: "Il vostro ERP ha migliorato la, ma ha dei bug"
                  Classificazione: Positiva

                [IMPORTANTE]: Tieni a mente che i feedback possono essere anche neutri.

          """
         },
        {"role": "user", "content": """
              "Il sistema presenta dei bug nell'interfaccia grafica, ma ha risolto tanti problemi"
        """}

# Il sistema CRM ha migliorato la gestione dei lead, ma l'interfaccia non è intuitiva,
# causando difficoltà di navigazione per il team.
# Si suggerisce una semplificazione dell'interfaccia con un layout organizzato e icone chiare per migliorare l'usabilità.
# Migliorare l'interfaccia potrebbe aumentare l'efficienza e facilitare la formazione di nuovi utenti.
# L'integrazione di funzionalità aggiuntive come un sistema di ricerca avanzato e strumenti di reporting personalizzabili migliorerebbe ulteriormente l'esperienza complessiva.
    ]
)

print(response.choices[0].message.content)

Classificazione: Positiva


In [24]:
# ONE-SHOT LEARNING
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_anthropic import ChatAnthropic

anthropic_llm = ChatAnthropic(
    model="claude-3-7-sonnet-20250219",
    api_key=userdata.get('ANTHROPIC_API_KEY')
)

system="CRM",
problem="Interfaccia lenta"
company_type="Retail"


# Creazione di un prompt template semplice
simple_template = PromptTemplate(
     input_variables=["ticket"],
     template="""
        Analizza questo ticket di supporto e determina priorità, modulo e tempo di risoluzione.

        ESEMPIO:
        Ticket: "Non riesco ad accedere al modulo HR da stamattina, dice 'credenziali scadute'"
        Analisi:
        - Priorità: Media
        - Modulo: HR/Gestione Utenti
        - Tempo stimato: 30 minuti



        Non motivare la tua scelta.

        ORA ANALIZZA: {ticket}
     """
 )

simple_chain = simple_template | anthropic_llm

simple_result = simple_chain.invoke({'ticket': """Il report delle fatture del mese scorso mostra totali errati.
        Alcune fatture sembrano duplicate nel modulo contabilità.
        Urgente per la chiusura trimestrale di domani."""})

# simple_result = simple_chain.invoke({'feedback': "Il vostro ERP ha risolto molti problemi, ma la reportistica è ancora lenta.", 'classificazione': "Neutro (aspetti positivi e negativi bilanciati)"})
print(simple_result.content)


- Priorità: Alta
- Modulo: Contabilità/Fatturazione
- Tempo stimato: 4 ore


In [47]:
# FEW-SHOT LEARNING

examples = [
    {
        'feedback': "Implementazione perfetta, tempi rispettati",
        'feedback_label': "P"
    },
    {
        'feedback': "Sistema funzionale ma setup laborioso",
        'feedback_label': "N"
    },
    {
        'feedback': "Troppe funzioni non operative, supporto lento",
        'feedback_label': "NEG"
    },
    {
        'feedback': "Buona integrazione ma prezzo elevato",
        'feedback_label': "N"
    },
]

example_formatter = PromptTemplate(
    input_variables=["feedback", "feedback_label"],
    template="{feedback} -> {feedback_label}"
)


few_shot_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_formatter,
    prefix="Classifica il feedback clienti come:  (positivo), N (neutro), NEG (negativo)",
    suffix="Ora classifica: {input_feedback}",
    input_variables=["input_feedback"]
)

(few_shot_template.get_prompts)

few_shot_chain =  few_shot_template | anthropic_llm
few_shot_result = few_shot_chain.invoke({'input_feedback': "Il sistema CRM ha migliorato la nostra gestione dei lead, ma l'interfaccia è ancora complessa e alcuni report non funzionano correttamente."})

print(few_shot_result.content)


Per classificare il feedback "Il sistema CRM ha migliorato la nostra gestione dei lead, ma l'interfaccia è ancora complessa e alcuni report non funzionano correttamente.", analizziamo le sue componenti:

Elementi positivi:
- Il sistema CRM ha migliorato la gestione dei lead

Elementi negativi:
- L'interfaccia è ancora complessa
- Alcuni report non funzionano correttamente

Questo feedback contiene sia aspetti positivi che negativi, con un elemento positivo significativo (miglioramento della gestione dei lead) bilanciato da due problemi specifici. Non è completamente negativo come il feedback etichettato NEG ("Troppe funzioni non operative, supporto lento"), ma presenta più criticità rispetto ai feedback classificati come N.

Considerando il bilanciamento tra l'aspetto positivo e i due aspetti negativi, la classificazione più appropriata è:

Label: N (neutro)


In [67]:
prompt_cot = """Analizza il seguente problema di performance ERP.
Ragiona passo-passo per identificare la causa principale
e suggerire soluzioni:

Passo 1: Consideriamo quando è iniziato il problema
Passo 2: Valutiamo cosa è cambiato nel sistema
Passo 3: Analizziamo il volume di dati coinvolti


Output aspettato:

Passo 1:
Passo 2:
Passo 3:

non aggiungere i nomi dei passi

Non aggiungere informazioni aggiuntive rispetto ai suddetti passi.
"""

prompt_tot = """Analizza il seguente problema di performance e rileva se si tratta di un ERP o un CRM.
Ragiona passo-passo, rilevando la procedura giusta tra quelle di sotto, per identificare la causa principale
e suggerire soluzioni:

PER CRM:
  1. Consideriamo quando è iniziato il problema
  2. Valutiamo cosa è cambiato nel sistema
  3. Analizziamo il volume di dati coinvolti

PER ERP:
  1. Analizziamo il DB
  2. Assicuriamoci che l'infra è up&running
  3. Verifichiamo che il servizio sia disponibile alla porta 443.


[IMPORTANT] Anche se i problemi ti sembrano di natura CRM, se la richiesta cita un ERP, trattalo con la procedura per gli ERP.

"""

user_prompt_1 = """
Il report mensile di vendite del CRM impiega 30 minuti per essere generato,
mentre prima richiedeva solo 5 minuti.
"""

user_prompt_2 = """
Un'azienda ERP nota che il 5% dei lead qualificati non si converte in opportunità.
I commerciali lamentano troppi passaggi nel sistema per registrare un'attività.
"""

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": prompt_tot},
        {"role": "user", "content": user_prompt_2}
    ]
)

print(response.choices[0].message.content)

Dato che il problema menziona esplicitamente un ERP, dovremmo trattarlo con la procedura dedicata agli ERP anche se il contesto potrebbe sembrare più simile a un CRM.

1. **Analizziamo il DB**: Iniziamo verificando se ci sono problemi con il database dell'ERP. Ciò include controllare la struttura del database per identificare eventuali inefficienze o malfunzionamenti che potrebbero ostacolare il processo di conversione da lead a opportunità. Possiamo esaminare le tabelle coinvolte nel processo di registrazione delle attività commerciali per accertarci che gli indici siano presenti e ottimizzati.

2. **Assicuriamoci che l'infra è up&running**: Dobbiamo verificare che l'infrastruttura su cui gira l'ERP sia completamente operativa. Ciò include controllare i server, la connettività di rete e rilevare eventuali problemi di configurazione che potrebbero causare ritardi o inefficienze nel sistema.

3. **Verifichiamo che il servizio sia disponibile alla porta 443**: Questo passaggio consiste n

In [ ]:

prompt_tot = """Analizza il problema e identifica la procedura passo-passo adatta in base alla problematica:

 Procedura per aziende B2B:
   - 1. Quali potrebbero essere le cause del basso tasso di conversione?
   - 2. Come il processo CRM attuale potrebbe influenzare questo?
   - 3. Quali metriche dovremmo analizzare per confermare le ipotesi?
   - 4. Quali soluzioni concrete possiamo implementare?


 Procedura per aziende ERP:
   - 1. Quali potrebbero essere le cause del poco coinvolgimento?
   - 2. Come il processo SISTEMA attuale potrebbe influenzare questo?
   - 3. Quali metriche abbiamo per le ipotesi?
   - 4. Quali soluzioni NON ci conviene intraprendere?
 """

prompt_react = """
   Problema: I dati degli ordini dal sito e-commerce non appaiono nel CRM.

   Segui il formato ReAct per diagnosticare:

   Thought: Verifica che la connessione al DB sia attiva.
   Action: Applica modifiche al DB se necessario
   Observation: Adesso i dati degli ordini sono presenti sul CRM?
   Thought: Il sistema funziona adesso?
   Action: Se il sistema funziona informa l'utente, in caso contrario riparti a debuggare il DB.
   ...

   Inizia l'analisi:
 """